In [1]:
import numpy as np

def generate_log2system(int_length, fraction_length):
  #system 2^(I.F)
  print ("generating table for 2^(I.F), I=%d, F=%d "%(int_length,fraction_length))
  step  =  2**(-fraction_length)
  power_table = np.arange(-2**int_length,  2**int_length, step)
  print ("power 2 table: ", list(power_table))
  table = list(map(lambda x: 2.0**x, power_table))
  print ("system 2^(%d.%d): "%(int_length,fraction_length), table)
  return table

weight_table = generate_log2system(2,1)


generating table for 2^(I.F), I=2, F=1 
power 2 table:  [-4.0, -3.5, -3.0, -2.5, -2.0, -1.5, -1.0, -0.5, 0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5]
system 2^(2.1):  [0.0625, 0.08838834764831845, 0.125, 0.1767766952966369, 0.25, 0.3535533905932738, 0.5, 0.7071067811865476, 1.0, 1.4142135623730951, 2.0, 2.8284271247461903, 4.0, 5.656854249492381, 8.0, 11.313708498984761]


In [2]:
import os
!nvidia-smi
!pip install transformers datasets
!pip install nlp

!git clone https://github.com/minhhn2910/QPyTorch
try:
  os.chdir('/content/QPyTorch')
except:
  pass
!git checkout posit-constant-generation

!pip install ./
!pip install ninja

try:
  os.chdir('/content/')
except:
  pass

import torch

if not torch.cuda.is_available():
  raise RuntimeError('Cannot run this cell without GPU runtime.')

import torch
import torch.nn as nn
import numpy as np




Mon Nov 18 17:46:32 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:5E:00.0 Off |                  N/A |
| 27%   31C    P8     3W / 250W |     20MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
device = 'cuda'
model_id = 'gpt2-large'
model = GPT2LMHeadModel.from_pretrained(model_id)
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)
from transformers import pipeline
from transformers import set_seed
from qtorch_plus.quant import posit_quantize, float_quantize, configurable_table_quantize
import transformers
transformers.logging.set_verbosity_error()



#final with 3 layers skipped :
full_table = np.array([6.59179688e-03, 2.19726562e-02, 4.02832031e-02, 6.25000000e-02,
                8.51440430e-02, 2.10937500e-01, 1.21093750e-01, 5.36407471e-01,
                1.56250000e-02, 6.25000000e-02, 1.25000000e-01, 1.64062500e-01,
                2.92968750e-01, 6.25000000e-01, 8.43750000e-01, 4.37500000e-01,
                1.00000000e+00, 2.11486816e+00, 1.50000000e+00, 1.25000000e+00,
                4.25000000e+00, 3.08789062e+00, 8.25000000e+00, 6.00000000e+00])

weight_table = full_table[:8]
act_table = full_table[8:]
weight_table = np.sort(weight_table)
act_table = np.sort(act_table)
print (len(weight_table))
print (weight_table)
print (act_table)

def linear_weight(input):
    # return input
    # return configurable_table_quantize(input, torch.tensor(weight_table,dtype = torch.float), scale= 1.0)
    return posit_quantize(input,nsize=8, es=1, scale = 1)
    # return float_quantize(input,exp=4, man=3, rounding="nearest")

def linear_activation(input):
    # return input
    # return configurable_table_quantize(input,torch.tensor(act_table, dtype=torch.float), scale= 1.0)
    return posit_quantize(input,nsize=8, es=1, scale = 1)
    # return float_quantize(input,exp=4, man=3, rounding="nearest")

def forward_pre_hook_linear(m, input):
    return (linear_activation(input[0]),)

model = model.to(device)
layer_count = 0
linear_layer_count = 0
op_count = 0
import transformers.modeling_utils as  modeling_utils
for name, module in model.named_modules():
  # print (type(module))
  if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear) or isinstance(module, modeling_utils.Conv1D):
    #print (name)
    layer_count = layer_count + 1
    #if ('lm_head' not in name and 'h.0' not in name):
    # print(f"\nBefore quantization - {name} weights:", module.weight.data.flatten()[:8])
    module.weight.data = linear_weight(module.weight.data)
    # print(f"After quantization - {name} weights:", module.weight.data.flatten()[:8])
    module.register_forward_pre_hook(forward_pre_hook_linear)

    if (isinstance(module, modeling_utils.Conv1D)):
      #print (module.weight.shape)
      op_count = op_count + module.weight.shape[0] *module.weight.shape[1]
    else:
      op_count = op_count + module.in_features*module.out_features
  elif isinstance(module, nn.Embedding):
    module.weight.data = linear_weight(module.weight.data)
    print('Quantizing the embeddings...')
print ("MAC operation count ", op_count)
print ("Layer count ", layer_count)


# from nlp import load_dataset

from datasets import load_dataset
test = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')
encodings = tokenizer('\n\n'.join(test['text']), return_tensors='pt')
#model = model.to(device)
def generate_text(model_new):
  text_generation = pipeline("text-generation", model=model_new, tokenizer=tokenizer, device = 0)
  #set_seed(42)
  prefix_texts = ["Machine learning is the study of",
                    "In the 19th century, the invention",
                    "A robot was created",
                    "One day I will"
                  ]
  for prefix_text in prefix_texts:
    #generated_text= text_generation(prefix_text, max_length=50, do_sample=False )[0]
    set_seed(42)
    generated_text= text_generation(prefix_text, max_length=50, num_return_sequences=3)
    print(generated_text[0]['generated_text'])
    print ("-----------------")
    print(generated_text[1]['generated_text'])
    print ("-----------------------------------")
    print(generated_text[2]['generated_text'])
    print ("-----------------------------------")
generate_text(model)

print ("\n FP32 ref : \n")
# model = GPT2LMHeadModel.from_pretrained(model_id).to(device)
# generate_text(model)


Using /home/yg9bq/.cache/torch_extensions/py310_cu118 as PyTorch extensions root...
Emitting ninja build file /home/yg9bq/.cache/torch_extensions/py310_cu118/quant_cpu/build.ninja...
Building extension module quant_cpu...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module quant_cpu...
Using /home/yg9bq/.cache/torch_extensions/py310_cu118 as PyTorch extensions root...


ninja: no work to do.


Detected CUDA files, patching ldflags
Emitting ninja build file /home/yg9bq/.cache/torch_extensions/py310_cu118/quant_cuda/build.ninja...
/home/yg9bq/miniconda3/envs/posit/lib/python3.10/site-packages/torch/utils/cpp_extension.py:1965: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Building extension module quant_cuda...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module quant_cuda...


ninja: no work to do.
8
[0.0065918  0.02197266 0.0402832  0.0625     0.08514404 0.12109375
 0.2109375  0.53640747]
[0.015625   0.0625     0.125      0.1640625  0.29296875 0.4375
 0.625      0.84375    1.         1.25       1.5        2.11486816
 3.08789062 4.25       6.         8.25      ]
Quantizing the embeddings...
Quantizing the embeddings...
MAC operation count  772117760
Layer count  145
Machine learning is the study of artificial intelligence algorithms that learn something by observing data. Machine learning helps humans and technology work better together.

What types of topics will I learn?

In each course, you'll learn relevant topics in machine learning
-----------------
Machine learning is the study of data algorithms and they will be taught in the context of our Master's Research Design. Their thesis will be about improving the capacity of computers to search a vast quantity of documents in different languages, both in the public domain and
-------------------------------

In [4]:
torch.cuda.empty_cache()

import torch
from tqdm import tqdm

max_length = model.config.n_positions
stride = 1024

lls = []
input_size = encodings.input_ids.size(1)
print("Input size:", input_size)

for i in tqdm(range(0, input_size, stride)):
    begin_loc = max(i + stride - max_length, 0)
    end_loc = min(i + stride, input_size)
    trg_len = end_loc - i

    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)
        log_likelihood = outputs[0] * trg_len
        # print(f"Log Likelihood for step {i}: {log_likelihood}")

    lls.append(log_likelihood)

if lls:  # Ensure lls is not empty
    ppl = torch.exp(torch.stack(lls).sum() / input_size)
    print("Perplexity:", ppl.item())
else:
    print("No log likelihoods calculated.")


Input size: 287644


100%|██████████| 281/281 [01:03<00:00,  4.43it/s]

Perplexity: 19.477630615234375
